This is a demo showing how to parse GraphViz files containing both HTML and String parameters.


In [1]:
import random
import sys 
import re
import dataclasses

import spot
import spot.gen as sg

import ply.lex as lex
import ply.yacc as yacc

from collections import namedtuple

from pathlib import Path

from graphviz import Digraph
from graphviz import Graph
from graphviz import Source   # Used to read existing dot files

from IPython.display import HTML, display

from GVParser import DotLexer

spot.setup(size='12,8')

----

### The Parser

In [2]:
class DotParser(object):

    tokens = DotLexer.tokens
    literals = DotLexer.literals
    reserved= DotLexer.reserved

    def __init__(self):
        self.lexer = DotLexer()
        self.parser = yacc.yacc(module=self,write_tables=False)

    def parse(self,dot):
        return self.parser.parse(dot, debug=False)
        
    def p_graph(self,p):
        ''' graph : DIGRAPH value body
                  '''
        p[0] = Graph(p[1],[p[2]],[p[3]])
#        p[0] = f'{p[1]} {p[2]} {p[3]}'
    #    print(f'graph:       {p[0]}')
    
    def p_body(self,p):
        '''body : bodystart statements bodyend
                '''
        p[0] = Graph("Graph",p[2],'')
#        print(f'Body:        {p[0]}')
    
    def p_bodystart(self,p):
        '''bodystart : '{' 
                     | '{' EOL
                '''
#        print(f'Bodystart')
    
    def p_bodyend(self,p):
        '''bodyend : '}' 
                     | '}' EOL
                '''
#        print(f'Bodystart')

    def p_statements(self,p):
        '''statements : statements statement
                      | statement
                      '''
        if len(p) == 3:
            p[1].append(p[2])
            p[0] = p[1]
        else:
            p[0] = [p[1]]
#        print(f'statements {p[0]}')

    def p_styles(self,p):
        '''styles : styles style
                  | style
                  '''
        if len(p) == 3:
            p[1].append(p[2])
            p[0] = list(p[1])
        else:
            p[0] = [p[1]]
#        print(f'Styles {p[0]}')
        
    def p_nodeStyle(self,p):
        '''statement : NODESTYLE LBRACKET styles RBRACKET EOL
                 '''
        p[0] = Style(p[3],[],'Nodestyle')
    
    def p_edgeStyle(self,p):
        '''statement : EDGESTYLE LBRACKET styles RBRACKET EOL
                 '''
        p[0] = Style(p[3],[],'Edgestyle')
    
    def p_node(self,p):
        '''statement : NUMBER EOL
                     | value EOL
                     '''
        p[0] = Text(p[1],'Node')
    
    def p_nodeStyled(self,p):
        '''statement : NUMBER LBRACKET styles RBRACKET EOL
                     | value LBRACKET styles RBRACKET EOL
                     '''
        p[0] = Node(p[1],p[3],"Node")
    
    def p_edgeConnect(self,p):
        '''statement : NUMBER CONNECTOR NUMBER EOL
                     | NUMBER CONNECTOR value EOL
                     | value CONNECTOR NUMBER EOL
                     | value CONNECTOR value EOL
                     '''
        p[0] = Edge(p[1],p[3])
        
    def p_edgeConnectStyled(self,p):
        '''statement : NUMBER CONNECTOR NUMBER LBRACKET styles RBRACKET EOL
                     | NUMBER CONNECTOR value LBRACKET styles RBRACKET EOL
                     | value CONNECTOR NUMBER LBRACKET styles RBRACKET EOL
                     | value CONNECTOR value LBRACKET styles RBRACKET EOL
                     '''
        p[0] = Edge(p[1],p[3],p[5])

    def p_styleLone(self,p):
        ''' statement : style EOL 
                      | style
                      '''
        p[0] = p[1]
#        print(f'style lone {p[0]}')
        
    def p_style(self,p):
        '''style : LABELLOC    '=' value 
                 | SHAPE       '=' value
                 | STYLE       '=' value
                 | COLOR       '=' value
                 | FILLCOLOR   '=' value
                 | FONTNAME    '=' value
                 | ARROWHEAD   '=' value
                 | ARROWSIZE   '=' NUMBER
                 | WIDTH       '=' NUMBER
                 | WIDTH       '=' value
                 | HEIGHT      '=' NUMBER
                 | HEIGHT      '=' value
                 | PERIPHERIES '=' NUMBER
                 | RANKDIR     '=' value
                 | CENTER      '=' value
                 | SIZE        '=' value
                 | LABEL       '=' value
                 | LABEL       '=' html
                 | TOOLTIP     '=' value
                 '''
        # print(f'style:       {p[0]}, {bcolors.BLUE}{type(p[0])}{bcolors.ENDC} {len(p)}')
        p[0] = Style(p[1],[p[3]])
#        print(f'style {p[0]}')

    def p_html(self,p):
        '''html : HTML_START htmlstring HTML_END
                '''
        p[0] = Style(p[2],'','HTML')
#        print(f'HTML {p[0]}')

    def p_htmlstring(self,p):
        '''htmlstring : htmlstring value
                      | htmlstring htmlstyle
                      | htmlstyle
                      | value
                      '''
        if len(p) == 3:
            p[1].append(p[2])
            p[0] = list(p[1])
        else:
            p[0] = [p[1]]
#        print(f'htmlstring {p[0]}')
        
    def p_htmlstyle(self,p):
        '''htmlstyle : HTML_START value style HTML_END
                     | HTML_START value HTML_END
                     '''
        if len(p) > 4:
            p[0] = Style(p[2],p[3],'HTML')
        else:
            p[0] = Style(p[2],'','HTML')
#        print(f'HTML style {p[0]}')
        
    def p_value(self,p):
        '''value : ID
                 | STRING
                     '''
        p[0] = Text(p[1])
#        print(f'Value {p[0]}')
        
    # Error rule for syntax errors
    def p_error(self,p):
        print("Syntax error in input!")
        print(f'{p}')


In [3]:
__file__ = "DC_Parser.ipynb"


In [4]:
def getSeed(rndSeed=False,printSeed=False):
#    seed = 96996 # generates a long diagram with multiple buchi labels
    seed = 79398 # generates a medium diagram with a graph label containing an & - not accounted for in yacc syntax
#    seed = 64165 # generates a graph title with brackets and html
    
    if rndSeed == True:
        seed = random.randrange(100000)

    if printSeed == True:
        print(f'Seed Used: {seed}')

    return(seed)

----
#### AST Structure

The following class is used for the top level of the AST.

In [5]:
class Graph:
    def __init__(self,type,value,children):
        self.type = type
        self.value = value
        self.children = children

    def display(self,tabCnt=0):
        for v in self.value:
            if type(v) is Graph:
                v.display(tabCnt=tabCnt+1)
            else:
                print(f'{chr(9) * tabCnt}{v}')
                
        for c in self.children:
            if type(c) is Graph:
                c.display(tabCnt=tabCnt+1)
            else:
                print(f'{chr(9) * tabCnt}{c}')
                
    def __repr__(self):
        return(f'Type {self.type}, Value {self.value}, Children {self.children}')

namedtuples will be used for most of the AST structure.

https://realpython.com/python-namedtuple/

In [6]:
Style = namedtuple("Style", ["Type", "Value", "name"],defaults=["Style"])
Text  = namedtuple("Text", ["Value", "name"],defaults=["Text"])
Edge  = namedtuple("Edge", ["From", "To", "Style", "name"],defaults=["","Edge"])
Node  = namedtuple("Node", ["NodeName", "Label", "name"],defaults=["Node"])


#### Formula Class

This class provides methods to create example GraphViz automata to demonstrate parsing to an AST.

One can either request a randomly generated formula from Spot, use a predefined one, or provide one as a string.

In [7]:
class Formula:

    def __init__(self,formulaStr,spotFormula,dOptions):
        self.formulaStr = formulaStr
        self.spotFormula = spotFormula
        self.formula = spotFormula.translate(*dOptions)
        self.formulaDot = self.formula.to_str('dot')
        
    @classmethod
    def getRndFormula(cls,dOptions,totalFormulas=10,rndSeed=True):
    
        seed = getSeed(rndSeed,printSeed=True)

        for formula in spot.randltl(3,totalFormulas,seed=seed, tree_size=(1, 40)):
            pass

        return cls(formula,formula,dOptions)

    @classmethod
    def getFixedFormula(cls,dOptions,fNumber=0):

        formula = [
                  '!(g | y )& r & X(r U( !(r|g) & y & X((y) U ((g & !(y | r)) & X(g U (!(g | y )& r))))))',
                  'G(r->X((r)U(y&X((y)U(g)))))',
                  '(p&X(q | (r & G(s))))U(t&F(u))',
                  '((!s & !p & t & r)  U (!r U ((!u & !t & (!p & X(p|s))))))',
                  'G((((p & !q) U((!p & q) | (r & G(s))))U(t &G(F(u) & X u))) & (t -> !p))',
                  '!(g | y )& r & X(r U( !(r|g) & y & X((y) U ((g & !(y | r)) & X(g U (!(g | y )& r))))))',
                  'G(((p & !q) U(!p & q)) U ((G t ) & X u))',
                  '(G !(c1 & c2)) & (GF c1) & (GF c2)',
                  'GF s &  GF p & GF q & GF !t & GF !u & G(t -> X!p)& G(u -> X!q)& G(p -> !t)& G(q -> !u)',
                  'G(!a | ((!b | X(!c U (d & Fe)) | X(c R !d)) U (c | G(!b | X(!c U (d & Fe)) | X(c R !d)))))',
                  'G((q & X F r) -> X(p -> (!r U s)) U r)',
                  'r & (GF r &  GF y & GF g & G(r -> X(g & !r & !y))& G(g -> X(!g & !r & y))& G(y -> X(!g & r & !y)) & G( g | r | y) )',
                  'r & (GF r &  GF y & GF g & G(r -> r U (g & !r & !y))& G(g -> g U(!g & !r & y))& G(y -> y U(!g & r & !y)) & G( g | r | y) )',
                  'r & ( G(r -> (r U g))& G(g -> (g U y)) & G(y -> (y U r)) & G(( g & !r & !y) | (!g & r & !y) | (!g & !r & y)))',
                  '( (( (p -> F( (u & q))) -> (!p -> !q)) & G( (q -> F(s)))) | G( (!t | !u)))',
                  'F(s)&F(p)&F(q)&(p->!t)&(q->!u)',
                  'G((p ∨ (q ∧ r)) -> X((s) U (t ∧ u)))',
                  'G((p & !t) -> X((q -> X(!q U (r & s & t))) U ((!r | !s) & u)))',
                  'G((p & !t) -> X((q -> X(!q U (r & s & t))) U (((!r & !s) | t | u) U (t & u))))',
                  'G((p & !t) -> X((!q & !u) -> X((q <-> (!r & s & t)) U (!s U ((q & !u) | (!q & (r | !t)))))))',
                  'G(F(p & (q | r)) -> X( X(!s)))',
                  'G((p & (q | r)) -> X(X(s)))',
                  'F(G(p -> X(q))) & G(r -> (F(!s)))',
                  'G(p -> (X(q) & G(r -> F(!s))))',
        ]

        return cls(formula[fNumber],spot.formula(formula[fNumber]),dOptions)

    @classmethod
    def setFormula(cls,dOptions,formula=''):

        return cls(formula,spot.formula(formula),dOptions)

    def getStartState(self):
        return self.formula.get_init_state_number()
        
    def showGraph(self,graphPrint,graphFile):
        dot = Source(self.formulaDot)
    
        if graphPrint['Print'] == True:
            display(dot)
    
        if graphPrint['Source'] == True:
            print(self.formulaDot)

        if graphPrint['Stats'] == True:
            display(self.formula.show_storage())
    
        if graphFile['save'] == True:
            dot.render(graphFile['name'], format=graphFile['format'])

    def hierarchyMap(self):
        display(self.spotFormula.show_mp_hierarchy())

    # Return the Manna and Pnueli Class
    def fClass(self):
        return spot.mp_class(self.spotFormula, 'wv')

    def acceptingRun(self):
        print(f'\n Spot generated accepting trace:')
        print(self.formula.accepting_run())

    def latex(self,title,color=''):
        print(f'\n{color}{"-"*30}\n   {title}\n{bcolors.ENDC}')
        print(f'{self.spotFormula:xc}') #See Spot documentation for further formatting: https://spot.lre.epita.fr/tut01.html
        print(f'{color}{"-"*30}')

    def toLaTeX(self,title,color=''):
        latexStr = ''
        dash = False
        for c in self.formulaStr:
            alpha = c.isalpha()
            cap   = c.isupper()
            if alpha == True and cap == True:
#                latexStr += f'\\mathsf{{{c}}}' #script G
                latexStr += f'{c}'
            elif c == '&' or c == chr(0x2227):
                latexStr += f'\\land '
            elif c == '!' or c == chr(0x00AC):
                latexStr += f'\\lnot '
            elif c == '|' or c == chr(0x2228):
                latexStr += f'\\lor '
            elif c == '-':
                dash = True
            elif dash == True:
                if c == '>':
                    latexStr += f'\\limplies'
                else:
                    latexStr += f'-{c}'
                dash = False
            else:
                latexStr += f'{c}'
        print(f'\n{color}{"-"*4} {title} {"-"*4}{bcolors.ENDC}')
        print(f'{latexStr}\n')
        print(f'{color}{"-"*30}{bcolors.ENDC}')
        
    def printSelf(self):

#        print(f'Starting formula:\n{self.formula}')
        display(self.spotFormula)
        
        

----
# Start of the demonstration program.

To control formula generation, these are the option parameters to set:

#### Specification Related Parameters
`formulaType` This defines where the specification comes from. The options are:
* 'Fixed' Used a fixed formula found in the Formula class.
* 'Random' Ask Spot for a random formula
* 'Given' Give the formula directly
  
`fixedFormulaNum ` If using a fixed formula, this parameter specifies which one. Currently allows selection 0-15  
`rndNumFormulas` If Spot is generating the formula, define how many to generate. The code will attempt to select an interesting one  
`startFormula` The specific formula to use  

`dOptions` All the formulas are sent to Spot to generate an automaton. This paramenter defines what type of automaton to create. The options are `Complete`, `Unambiguous`, `tgba`. See the Spot documentation for more info.

`rndSeed=True` For repeatability, set this to false. when set to True, the seed used is defined in getSeed().  

In [8]:
formulaType = 'Given' #'Fixed', 'Random', 'Given'
startFormula = 'G(r->X((r)U(y&X((y)U(g)))))'
startFormula = '(l ↔ p) ∧ F(¬p)'
startFormula = '(l ↔ p) ∧ F(¬p) & G !((l & !p) | (!l & p))'
startFormula = 'G(l ↔ p) ∧ F(¬p)'
startFormula = 'G((t ↔ y) ∧ (t → p))'
startFormula = '(G((l ↔ p) ∧ F(¬p)) ∧ G((t ↔ y) ∧ (t → p)))'
#startFormula = 'G(((l ↔ p) ∧ ((t ↔ y) ∧ (t → p)) ∧ F(¬p)))'
startFormula = '(G((l ↔ p) ∧ F(¬p)) ∧ G((t → (y ∧ X¬y)) ∧ (t → X(b U b)) ∧ (¬(b ∧ t) → X¬b) ∧ (t → p)))'
#startFormula = 'G(l <-> p) & G(t -> (p & y & X (!t & !y))) & F(!p)'
#startFormula = 'G(l <-> p) & G((!y U (p & t & X!y))) & F(!p)'
#startFormula = 'GF r &  GF y & GF g & G(r -> X(g & !r & !y)) & G(g -> X(y & !r & !g))& G(y -> X(r & !g & !y))'
#startFormula = 'G((r -> Xg) ∧ (g -> Xy) ∧ (y -> Xr)) ∧ G(¬(r ∧ y) ∧ ¬(r ∧ g) ∧ ¬(y ∧ g))'
#startFormula = 'G((r -> Xg) ∧ (g -> Xy) ∧ (y -> Xr)) ∧ G(¬(r ∧ y) ∧ ¬(r ∧ g) ∧ ¬(y ∧ g)) ∧ G(r -> XXg ∧ y -> XXg)'
startFormula = 'G((r -> Xg) ∧ ((g & Xg) -> Xy) ∧ (y -> Xr)) ∧ G(¬(r ∧ y) ∧ ¬(r ∧ g) ∧ ¬(y ∧ g))'
#startFormula = 'G((r -> Xg) ∧ (g -> Xr)) ∧ G(¬(r ∧ g))'
#startFormula = 'G(r -> (Xg -> (Xg -> Xr))) ∧ G(¬(r ∧ g)) & G(r | g)'
#startFormula = 'G((p <-> g) & F(!p) & ((r -> X(y & X(!y))) & (y <-> r)) & (r & !escape -> (b U (!b | !p))) & G(!b | !r))'

#startFormula = 'G(a -> X F b)'
#startFormula = 'G(a -> X G ( b -> X F (c -> X F d)))'

# These equations come from "Li, R., Gurushankar, K., Heule, M. J., & Rozier, K. Y. (2023, October). What's in a Name? Linear Temporal Logic Literally Represents Time Lines. 
# In 2023 IEEE Working Conference on Software Visualization (VISSOFT) (pp. 73-83). IEEE."
#startFormula = 'G((a & b) -> X !b)' # Figure 8
#startFormula = 'c & (F G a U X(G b & (!a U F a)))' # Figure 9
#startFormula = 'G((p & X !p) | (!p & X p))' # Figure 10

#startFormula = 'G((a U (b U (c)))&(!(a & b) & !(a & c) & !(b & c)))'
#startFormula = 'G((a & X !(a & b) | (!a & X (!a & !b)) | (b & X !(a & b) | (!b & X (!a & !b)))))'
#startFormula = 'G(((a & X (!a & b)) | (b & X (!b & a))) & !(a & b) & !(!a & !b))' # RE (ab)*
#startFormula = 'G(((a & X (!a & b & !c)) | (b & X (!b & !a & c)) | (c & X (!c & !b & a))) & !(a & b & c) & !(!a & !b & !c))' # RE (ab)*
#startFormula = 'G(((a & X (!a &  b & !c & !d)) |(b & X (!a & !b &  c & !d)) |(c & X (!a & !b & !c &  d)) |(d & X ( a & !b & !c & !d)))&!(a & b & c & d)&!(!a & !b & !c & !d))' # RE (abcd)*
#startFormula = 'a & X G((((a & X (!a & b & !c)) | (b & X (!b & !a & c)) | (c & X (!c & !b & a))) & G(!(a & b) & !(a & c) & !(b & c))))' # RE a(abc)* 
#startFormula = 'a & X G(((a -> Xb) & (b -> X(c|d)) & (c -> Xa) & (d -> Xb)) & G(!(a & b) & !(a & c) & !(a & d) & !(b & c) & !(b & d) & !(c & d) & (a | b | c |d)))' # RE a(a|b|c|d)(ab(db)*c)* 

#startFormula = 'G((r -> Xg) ∧ (g -> Xy) ∧ (y -> Xr) & G(r | g | y)) ∧ G(¬(r ∧ y) ∧ ¬(r ∧ g) ∧ ¬(y ∧ g) & (F r & F g & F y)) ' # Basic traffic light
#startFormula = 'G(p <-> g) & F(!p) & (g & t <-> y) & GF(p &( a & Xc)) & GF(p & b) & G((a->!b) & (b->!a))' 
#startFormula = 'G(p <-> g) & F(!p) & GF(p &( a & Xc)) & GF(p & b) & G((a->!b) & (b->!a))' 
#startFormula = 'G(g <-> p) & G(b U (!b | a)) & G(a -> X c) & F(¬p) & G(a->p) & G(b->p) & G(c->p) & GFa & GFb & GFc & GF(p & !a & !b & !c) & G(a -> !b) & G(b->!a)'
#startFormula = 'G((a & b) -> X(a & c)) & G((a & c) -> X(d & !a))'
#startFormula = 'G(((G(!s) & (q & r)) -> (s U X((t & u)))))'
#startFormula = 'G((p | (q& r)) ->  X((s) U ( t& u)))'
#startFormula = 'F(p & q)'
#startFormula = 'p'
#startFormula = 'GF((p | q) & (p -> Xq) & (q -> Xp)) & G!(p & q)'
#startFormula = 'G(F(((X((nsg & ewr) & X((nsy & ewr) & X (nsr & ewg))))) | ((X((ewg & nsr) & X((ewy & nsr) & X (ewr & nsg)))))) & F nsg & F nsy & F ewg & F ewy) & G!(nsr & nsg) & G!(nsr & nsy) & G!(nsg & nsy) & G((ewr & (nsg | nsy)) | (nsr & (ewg | ewy))) & G!(ewr & ewy) & G!(ewr & ewg) &G!(ewg & ewy) '
#startFormula = 'GF((ag R ar)| (bg R br)) & GF ag & GF bg & GF ar & GF br & G!(ag & bg) '
#startFormula = 'GF(r & a & o -> !h) & GF(rbl -> rc & b) & GF(rw -> (r & b & d) & X !rw) & GF rw & G(!rw & rc)'
#startFormula = 'GF((as | (ap & lr)) & X(G!(as | ap) U (vs | (vp & av)))) & G!(ap & vp) & G!(ap & !lr) & G!(vp & !av)'
#startFormula = 'F((p -> G(q))&(r->X(s)))'
startFormula = 'F(BF & X(SIK & X(PIK & X(VC & X(PH & X(PH & X(IAH & X(II & X(MM))))))))) '

rndSeed=False

rndNumFormulas = 10
fixedFormulaNum = 4
dOptions = ['Complete', 'Buchi', 'state-based', 'det','high'] # First option must be one of: 'Complete' 'Unambiguous' 'tgba'
#dOptions = ['Complete', 'Buchi', 'det','high'] # First option must be one of: 'Complete' 'Unambiguous' 'tgba'

saveDir = 'testRun1'
path = Path(__file__).parent.absolute().joinpath(saveDir)
path.mkdir(parents=True, exist_ok=True)


In [9]:
def buildAutomaton(formula: Formula):

    parser = DotParser()
#    results = parser.parse(formula.formulaDot)

#    formula = dotdata = 'digraph MONA_DFA { rankdir = LR; center = true; size = "7.5,10.5"; edge [fontname = Courier]; node [height = .5, width = .5]; node [shape = doublecircle]; 2; node [shape = circle]; 0; 1; node [shape = box]; init [shape = plaintext, label = ""]; init -> 0; 0 -> 1 [label=<X<font color="#1F78B4"><I><B>y</B>⓿</I>z</font>)>]; 1 -> 2 [label="X"]; 2 -> 2 [label="X"];} '
    results = parser.parse(formula)

    return(results)
    

In [14]:
formulaA ='digraph MONA_DFA { rankdir = LR; center = true; size = "7.5,10.5"; edge [fontname = Courier]; node [height = .5, width = .5]; node [shape = doublecircle]; 2; node [shape = circle]; 0; 1; node [shape = box]; init [shape = plaintext, label = ""]; init -> 0; 0 -> 1 [label=<X<font color="#1F78B4"><I><B>y</B>⓿</I>z</font>)>]; 1 -> 2 [label="X"]; 2 -> 2 [label="X"];} '

if formulaType == 'Random':
    totalFormulas = 10
    formula = Formula.getRndFormula(dOptions,totalFormulas=rndNumFormulas,rndSeed=rndSeed)
elif formulaType == 'Fixed':
    formula = Formula.getFixedFormula(dOptions,fNumber=fixedFormulaNum)
else:
    formula = Formula.setFormula(dOptions,startFormula)

graph = formulaA
graph = formula.formulaDot

# This is a little debug code that allows me to limit the length of the input to the parser so there's less text to churn through after throwing an error. It also helps to isolate the erroneous line.
p = re.compile('digraph [^\n;]*[\n;][^\n;]*[\n;][^\n;]*[\n;][^\n;]*[\n;][^\n;]*[\n;][^\n;]*[\n;][^\n;]*[\n;][^\n;]*[\n;][^\n;]*[\n;][^\n;]*[\n;][^\n;]*[\n;][^\n;]*[\n;][^\n;]*')
g = p.match(graph).group() + ';\n}'
g = graph

print(g)
auto = buildAutomaton(g)

print('\n')
auto.display()



digraph "" {
  rankdir=LR
  label=<[Büchi]>
  labelloc="t"
  node [shape="ellipse",width="0.5",height="0.5"]
  node [style="filled", fillcolor="#ffffaa"]
  fontname="Lato"
  node [fontname="Lato"]
  edge [fontname="Lato"]
  size="12,8" edge[arrowhead=vee, arrowsize=.7]
  I [label="", style=invis, width=0]
  I -> 0
  0 [label=<253>]
  0 -> 0 [label=<!BF>]
  0 -> 1 [label=<BF>]
  1 [label=<254>]
  1 -> 0 [label=<!BF &amp; !SIK>]
  1 -> 1 [label=<BF &amp; !SIK>]
  1 -> 2 [label=<!BF &amp; SIK>]
  1 -> 3 [label=<BF &amp; SIK>]
  2 [label=<255>]
  2 -> 4 [label=<!BF &amp; PIK>]
  2 -> 5 [label=<BF &amp; PIK>]
  2 -> 0 [label=<!BF &amp; !PIK>]
  2 -> 1 [label=<BF &amp; !PIK>]
  3 [label=<256>]
  3 -> 4 [label=<!BF &amp; PIK &amp; !SIK>]
  3 -> 5 [label=<BF &amp; PIK &amp; !SIK>]
  3 -> 6 [label=<!BF &amp; PIK &amp; SIK>]
  3 -> 7 [label=<BF &amp; PIK &amp; SIK>]
  3 -> 0 [label=<!BF &amp; !PIK &amp; !SIK>]
  3 -> 1 [label=<BF &amp; !PIK &amp; !SIK>]
  3 -> 2 [label=<!BF &amp; !PIK &amp; SIK>

Generating LALR tables


In [11]:
print(auto)

Type digraph, Value [Text(Value='""', name='Text')], Children [Type Graph, Value [Style(Type='rankdir', Value=[Text(Value='LR', name='Text')], name='Style'), Style(Type='label', Value=[Style(Type=[Text(Value='[Büchi]', name='Text')], Value='', name='HTML')], name='Style'), Style(Type='labelloc', Value=[Text(Value='"t"', name='Text')], name='Style'), Style(Type=[Style(Type='shape', Value=[Text(Value='"ellipse"', name='Text')], name='Style'), Style(Type='width', Value=[Text(Value='"0.5"', name='Text')], name='Style'), Style(Type='height', Value=[Text(Value='"0.5"', name='Text')], name='Style')], Value=[], name='Nodestyle'), Style(Type=[Style(Type='style', Value=[Text(Value='"filled"', name='Text')], name='Style'), Style(Type='fillcolor', Value=[Text(Value='"#ffffaa"', name='Text')], name='Style')], Value=[], name='Nodestyle'), Style(Type='fontname', Value=[Text(Value='"Lato"', name='Text')], name='Style'), Style(Type=[Style(Type='fontname', Value=[Text(Value='"Lato"', name='Text')], name

In [12]:
print(formula.formulaDot)

digraph "" {
  rankdir=LR
  label=<[Büchi]>
  labelloc="t"
  node [shape="ellipse",width="0.5",height="0.5"]
  node [style="filled", fillcolor="#ffffaa"]
  fontname="Lato"
  node [fontname="Lato"]
  edge [fontname="Lato"]
  size="12,8" edge[arrowhead=vee, arrowsize=.7]
  I [label="", style=invis, width=0]
  I -> 0
  0 [label=<253>]
  0 -> 0 [label=<!BF>]
  0 -> 1 [label=<BF>]
  1 [label=<254>]
  1 -> 0 [label=<!BF &amp; !SIK>]
  1 -> 1 [label=<BF &amp; !SIK>]
  1 -> 2 [label=<!BF &amp; SIK>]
  1 -> 3 [label=<BF &amp; SIK>]
  2 [label=<255>]
  2 -> 4 [label=<!BF &amp; PIK>]
  2 -> 5 [label=<BF &amp; PIK>]
  2 -> 0 [label=<!BF &amp; !PIK>]
  2 -> 1 [label=<BF &amp; !PIK>]
  3 [label=<256>]
  3 -> 4 [label=<!BF &amp; PIK &amp; !SIK>]
  3 -> 5 [label=<BF &amp; PIK &amp; !SIK>]
  3 -> 6 [label=<!BF &amp; PIK &amp; SIK>]
  3 -> 7 [label=<BF &amp; PIK &amp; SIK>]
  3 -> 0 [label=<!BF &amp; !PIK &amp; !SIK>]
  3 -> 1 [label=<BF &amp; !PIK &amp; !SIK>]
  3 -> 2 [label=<!BF &amp; !PIK &amp; SIK>

----
#### Code for testing just the Lexer

In [13]:
# Build the lexer
lexer = DotLexer()
lexer.build()
g = graph

# Give the lexer some input
lexer.test(g)
print(g)

LexToken(DIGRAPH,'digraph',1,0), DIGRAPH
LexToken(STRING,'""',1,8), STRING
LexToken({,'{',1,11), {
LexToken(EOL,'\n',1,12), EOL
LexToken(RANKDIR,'rankdir',1,15), RANKDIR
LexToken(=,'=',1,22), =
LexToken(ID,'LR',1,23), ID
LexToken(EOL,'\n',1,25), EOL
LexToken(LABEL,'label',1,28), LABEL
LexToken(=,'=',1,33), =
LexToken(HTML_START,'<',1,34), HTML_START
LexToken(STRING,'[Büchi]',1,35), STRING
LexToken(HTML_END,'>',1,42), HTML_END
LexToken(EOL,'\n',1,43), EOL
LexToken(LABELLOC,'labelloc',1,46), LABELLOC
LexToken(=,'=',1,54), =
LexToken(STRING,'"t"',1,55), STRING
LexToken(EOL,'\n',1,58), EOL
LexToken(NODESTYLE,'node',1,61), NODESTYLE
LexToken(LBRACKET,'[',1,66), LBRACKET
LexToken(SHAPE,'shape',1,67), SHAPE
LexToken(=,'=',1,72), =
LexToken(STRING,'"ellipse"',1,73), STRING
LexToken(WIDTH,'width',1,83), WIDTH
LexToken(=,'=',1,88), =
LexToken(STRING,'"0.5"',1,89), STRING
LexToken(HEIGHT,'height',1,95), HEIGHT
LexToken(=,'=',1,101), =
LexToken(STRING,'"0.5"',1,102), STRING
LexToken(RBRACKET,']',1